<a href="https://colab.research.google.com/github/MatheusABomfim/Mest_MBL2_ML_HTLV/blob/main/MEST_HTLV%2BMBL2%2BCLINICAL_Hyperparameters_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading data and packages**

**Packages install**

In [2]:
# Att Scikit-learn, imbalanced learn and scikit-optimize
!pip install scikit-learn==1.0.2
!pip install imbalanced-learn
!pip install scikit-optimize

  Using cached scikit-learn-1.0.2.tar.gz (6.7 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [66 lines of output]
      Partial import of sklearn during the build process.
      setup.py:128: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
        from numpy.distutils.command.build_ext import build_ext  # noqa
      INFO: No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
      Traceback (most recent call last):
        File "c:\Users\mathe\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_proces

   ---------------------------------------- 0.0/107.7 kB ? eta -:--:--
   ----------- ---------------------------- 30.7/107.7 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 107.7/107.7 kB 1.5 MB/s eta 0:00:00


**Libraries import**

In [1]:
# Import necessary libraries and packages
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

# sklearn imports
from sklearn.model_selection import (
    train_test_split, cross_val_score, LeaveOneOut, cross_validate,
    learning_curve
)

from sklearn.ensemble import (
    ExtraTreesClassifier, RandomForestClassifier,
    AdaBoostClassifier, GradientBoostingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay, classification_report,
    recall_score, accuracy_score, roc_curve, auc, make_scorer,
    precision_score, f1_score
)
from sklearn.preprocessing import (
    LabelEncoder, MinMaxScaler, MaxAbsScaler, QuantileTransformer,
    RobustScaler, StandardScaler
)
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

# imbalanced-learn imports
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN

# joblib imports
from joblib import dump, load

# scikit-optimize imports
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# statsmodels imports
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# openpyxl imports
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# ipywidgets imports
from ipywidgets import interact, widgets

# Ignore warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import make_scorer
import warnings
import os
from joblib import dump


**Data import**

In [2]:
# Path to the CSV file containing the DataFrame
combined_data_file = '..\\datasets\\df_combined.csv'

# Loading the DataFrame directly from the CSV file
df_combined = pd.read_csv(combined_data_file)

# Normalizando apenas a coluna "Idade"
scaler = MinMaxScaler()
df_combined['Idade'] = scaler.fit_transform(df_combined[['Idade']])

# Displaying the first rows of the DataFrame with the “Age” column normalized
display(df_combined.head())


,Sexo,Idade,MBL-550,MBL -221,MBL EXON,Sintomas urinários,Dor,Transtorno/sintomas neurológicos,HAM/TSP
0,0,0.758621,1,0,2,2,4,5,0
1,1,0.137931,0,0,1,0,0,0,0
2,0,0.517241,2,1,0,0,0,0,0
3,0,0.206897,1,0,0,0,0,0,0
4,0,0.293103,1,1,1,0,0,0,0


In [3]:
X = df_combined.drop(columns=['HAM/TSP']) 
y = df_combined['HAM/TSP'] 

# Hyperparametes tuning by bayeasian optimization

**Search for the best hyperparameters and saved definitions**

In [4]:
# Define a random state for reproducibility
RANDOM_STATE = 42

# Ignore warnings
warnings.filterwarnings('ignore')

balancing_methods = [
    RandomUnderSampler(random_state=RANDOM_STATE),
]

# Common settings for all models
common_settings = {
    'cv': LeaveOneOut(),
    'n_iter': 10,
    'n_jobs': -1,
    'verbose': 0,
}

# Bayesian search function
def bayes_search(model, space):
    bs = BayesSearchCV(
        model, space,
        n_iter=common_settings['n_iter'],
        cv=common_settings['cv'],
        random_state=RANDOM_STATE,
        scoring=make_scorer(geometric_mean_score)
    )
    return bs

# Model space and hyperparameters
model_space = {
    'DT': {
        'model': DecisionTreeClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__max_depth': Integer(3, 5),
            'classifier__min_samples_split': Integer(5, 10),
            'classifier__min_samples_leaf': Integer(1, 5),
            'classifier__criterion': Categorical(['gini', 'entropy']),
            'classifier__ccp_alpha': Real(0.0, 0.1),
            }
    },
    'GB': {
        'model': GradientBoostingClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__n_estimators': Integer(50, 500),
            'classifier__learning_rate': Real(0.01, 1.0, 'log-uniform'),
            'classifier__max_depth': Integer(1, 10),
            'classifier__min_samples_split': Integer(2, 20),
            'classifier__min_samples_leaf': Integer(1, 20),
            'classifier__subsample': Real(0.5, 1.0, 'log-uniform'),
            'classifier__loss': Categorical(['deviance', 'exponential']),
        }
    },
    'LR': {
        'model': LogisticRegression(random_state=RANDOM_STATE),
        'space': {
            'classifier__C': Real(0.0001, 1000, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__solver': Categorical(['liblinear', 'sag', 'saga']),
            'classifier__class_weight': Categorical(['balanced', None]),
            'classifier__tol': Real(0.0001, 0.001),
        }
    },
    'MLP': {
        'model': MLPClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__hidden_layer_sizes': Integer(2, 16),
            'classifier__activation': Categorical(['logistic', 'tanh', 'relu']),
            'classifier__learning_rate': Categorical(['constant', 'adaptive']),
            'classifier__learning_rate_init': Real(0.001, 0.1, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__solver': Categorical(['sgd', 'adam']),
            'classifier__momentum': Real(0.1, 0.9, 'log-uniform'),
        }
    },
    'NB': None,
    
    'RF': {
        'model': RandomForestClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__n_estimators': Integer(50, 500),
            'classifier__max_depth': Integer(3, 10),
            'classifier__min_samples_split': Integer(2, 10),
            'classifier__min_samples_leaf': Integer(1, 5),
            'classifier__criterion': Categorical(['gini', 'entropy']),
            'classifier__max_samples': Real(0.5, 1.0, 'log-uniform'),
            'classifier__class_weight': Categorical(['balanced', 'balanced_subsample']),
        }
    },
    'SVM': {
        'model': SVC(probability=True, random_state=RANDOM_STATE),
        'space': {
            'classifier__kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
            'classifier__gamma': Real(0.001, 1000),
            'classifier__degree': Integer(2, 5),
            'classifier__coef0': Real(0.0001, 1, 'log-uniform'),
            'classifier__C': Real(0.001, 1000),
            'classifier__tol': Real(0.00001, 0.1, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__class_weight': Categorical([None, 'balanced']),
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'space': {
            'classifier__n_neighbors': Integer(1, 20),
            'classifier__weights': Categorical(['uniform', 'distance']),
            'classifier__algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
            'classifier__leaf_size': Integer(20, 50),
            'classifier__p': Integer(1, 2),
        }
    },
    'AdaBoost': {
        'model': AdaBoostClassifier(random_state=RANDOM_STATE),
        'space': {
            'classifier__n_estimators': Integer(50, 500),
            'classifier__learning_rate': Real(0.01, 1.0, 'log-uniform'),
        }
    },
    'LSVM': {
        'model': LinearSVC(random_state=RANDOM_STATE),
        'space': {
            'classifier__C': Real(0.0001, 1000, 'log-uniform'),
            'classifier__tol': Real(0.00001, 0.1, 'log-uniform'),
            'classifier__max_iter': Integer(200, 2000),
            'classifier__class_weight': Categorical([None, 'balanced']),
        }
    },
}

# Template saving directory
save_dir ='..\\models\\'

# Iterating over the models in the search space
for model_name in model_space:
    model = model_space[model_name]['model']
    space = model_space[model_name]['space']

    pipeline = Pipeline([
        ('normalization', MinMaxScaler()),
        ('balancing', RandomUnderSampler(random_state=RANDOM_STATE)),
        ('classifier', model)
    ])

    # Check that the search space is not empty
    if space:
        bs = bayes_search(pipeline, space)
        bs.fit(X, y)

        print(f'Model: {model_name}')
        print(f'Melhor valor de G-mean: {bs.best_score_}')
        print(f'Melhores parâmetros: {bs.best_params_}')

        # Saving the best model
        model_path = os.path.join(save_dir, f'best_{model_name}_model.joblib')
        dump(bs.best_estimator_, model_path)

    else:
        # Train the model directly if there are no hyperparameters to adjust
        pipeline.fit(X, y)
        print(f'Model: {model_name}')
        print('Modelo treinado diretamente sem busca Bayesiana.')

        # Saving the trained model
        model_path = os.path.join(save_dir, f'best_{model_name}_model.joblib')
        dump(pipeline, model_path)


Model: DT
Melhor valor de G-mean: 0.9215686274509803
Melhores parâmetros: OrderedDict([('balancing', RandomUnderSampler(random_state=42)), ('classifier__ccp_alpha', 0.08178300196637517), ('classifier__criterion', 'entropy'), ('classifier__max_depth', 4), ('classifier__min_samples_leaf', 3), ('classifier__min_samples_split', 6)])
Model: GB
Melhor valor de G-mean: 0.8496732026143791
Melhores parâmetros: OrderedDict([('balancing', RandomUnderSampler(random_state=42)), ('classifier__learning_rate', 0.022066991249460103), ('classifier__loss', 'exponential'), ('classifier__max_depth', 8), ('classifier__min_samples_leaf', 11), ('classifier__min_samples_split', 4), ('classifier__n_estimators', 390), ('classifier__subsample', 0.9154990875906714)])
Model: LR
Melhor valor de G-mean: 0.8235294117647058
Melhores parâmetros: OrderedDict([('balancing', RandomUnderSampler(random_state=42)), ('classifier__C', 8.023246075377976), ('classifier__class_weight', None), ('classifier__max_iter', 948), ('class